https://gdcoder.com/sentiment-clas/

To jest zamiast tej części poprzedzającej "Text data preprocessing"

Zacznijcie pisać z tutka w kolejnej linijce poniżej tych co są tutaj (tylko wcześniej załadujcie te komórki aby wstępnie obrobić dataset)

import bibliotek które wykorzystamy do podziału naszego datasetu na zbiór uczący i testowy

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

%matplotlib inline

import re, string
from sklearn.feature_extraction.text import CountVectorizer

Ładujemy dataset do zmiennej

In [2]:
df = pd.read_csv("IMDB Dataset.csv")

Sprawdzamy sobie jak wygląda dataset poprzez funkcję head()

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Zamieniamy wartości tekstowe na liczbowe

In [4]:
df.sentiment= df.sentiment.replace(['positive','negative'], [1,0])

Tworzymy zbiór wartości (x) oraz etykiet (y)

In [5]:
y = df.sentiment.reset_index(drop=True,)
x = df.drop('sentiment', axis=1).reset_index(drop=True)

In [6]:
x.head()

,review
0,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...
2,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is..."


In [7]:
y.head()

0    1
1    1
2    1
3    0
4    1
Name: sentiment, dtype: int64

Dzielimy zbiory wartosci i etykiet na zbiory do trenowania i testowania naszego klasyfikatora

In [8]:
trn, val, trn_y, val_y = train_test_split(x,y, test_size=0.2, random_state=50)

In [9]:
len(trn), len(val), len(trn_y), len(val_y)

(40000, 10000, 40000, 10000)

Liczymy ilość pozycji 

In [10]:
trn_y = trn_y.values
type(trn_y)


numpy.ndarray

In [11]:
trn = trn['review'].astype(str).values.tolist()

In [12]:
val_y = val_y.values
type(val_y)

numpy.ndarray

In [13]:
val = val['review'].astype(str).values.tolist()

# Tokenizacja - podział na pojedyncze słowa, bez znaczenia kolejności tych słów

In [14]:
%matplotlib inline

import re, string
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

#create term documetn matrix
veczr = CountVectorizer(tokenizer=tokenize)

Tworzenie dokumentu do zapisu podzielonych słów

In [16]:
trn_term_doc = veczr.fit_transform(trn)
# Important: Use same vocab for validation set
val_term_doc = veczr.transform(val)

In [17]:
max_df = 0.9
min_df = 5

Wyrzucenie słów o bardzo wysokiej częstotliwości (np. "i", "lub"), oraz słów rzadkich (powtórzonych mniej niż 5 razy)

In [18]:
trn_term_doc

<40000x93289 sparse matrix of type '<class 'numpy.int64'>'
	with 5938628 stored elements in Compressed Sparse Row format>

Liczymy wymiary arkusza oraz ilość elementów

In [19]:
trn_term_doc[5] #142 stored elements
w0 = set([o.lower() for o in trn[5].split(' ')]); w0
len(w0)
# length is 78 which is pretty similar to 83. the difference
# can be attributed to the fact that I didn’t use a real tokenizer. 

142

In [20]:
vocab = veczr.get_feature_names()
print(len(vocab))
vocab[5000:5005]

93289


['aproved', 'aprox', 'apsion', 'apt', 'aptitude']

Wyświetla słowa pod pozycjami od 5000 do 5005.

In [27]:
veczr.vocabulary_['aproved']

5000

Wyświetla pozycję słowa 'aproved'

In [29]:
trn_term_doc[0,6297]
# word 'absurb' appears twice in the first document

0

Przeszukiwanie słów

In [30]:
vocab[4050]

'anastas'

In [31]:
veczr.vocabulary_['arching']

5000

In [32]:
trn_term_doc[0,4050]
# word 'arching' does not appear in the first document

0

# Implementacja Naiwnego klasyfikatora Bayesa

In [34]:
def pr(y_i):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1) # plus 1 for the row of ones we added

x=trn_term_doc
y=trn_y

In [40]:
r = np.log(pr(1)/pr(0))
b = np.log((y==1).mean() / (y==0).mean())

SyntaxError: invalid syntax (<ipython-input-40-87b4e6b6455a>, line 2)

In [44]:
r.shape,val_term_doc.shape

pre_preds = val_term_doc @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

0.8304

Wynik podstawowego modelu

In [45]:
x=trn_term_doc.sign()
r = np.log(pr(1)/pr(0))

pre_preds = val_term_doc.sign() @ r.T + b #sign binarize
preds = pre_preds.T>0
(preds==val_y).mean()

0.8462

Usprawnienie modelu poprzez binaryzację wartości słów (brak podwójnego liczenia)